# Ray cluster

Ray is a framework for distributed computing in Python. It is growing in popularity.

Please note that the Ray project provides its own docker image `rayproject/ray`, however, installing dependencies manually adds to the intuition.

#### Create a docker image which has all necessary software installed

```Dockerfile
FROM continuumio/miniconda3
RUN apt update && apt install -y iputils-ping iproute2
RUN pip install "ray[all]"
``

In [ ]:
%%time
!docker build -t test-ray-image .

#### Create a simulated network

In [ ]:
!docker network rm simulated-cluster

In [ ]:
!docker network create simulated-cluster

#### Start a docker container 
Scheduler at port 6379 and UI at port 8265

In [ ]:
!docker run \
    -dit \
    --network simulated-cluster \
    -p 6379:6379 -p 8265:8265 -p 10001:10001 -p 10002:10002 \
    --name ray-head \
test-ray-image \
    ray start --head --node-ip-address=0.0.0.0 --dashboard-host=0.0.0.0 --disable-usage-stats --block

In [1]:
!docker run \
    -dit \
    -p 6379:6379 -p 8265:8265 -p 10001:10001 -p 10002:10002 \
    --name ray-head \
test-ray-image \
    ray start --head --node-ip-address=0.0.0.0 --dashboard-host=0.0.0.0 --disable-usage-stats --block

dff21f978052157439b5ac01738321542d31bb012a7d5a365b973536b04abfcf


In [3]:
!docker logs ray-head

Usage stats collection is disabled.

Local node IP: 0.0.0.0
2025-03-04 01:33:23,637	WARNING services.py:2072 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.66gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.

--------------------
Ray runtime started.
--------------------

Next steps
  To add another node to this Ray cluster, run
    ray start --address='0.0.0.0:6379'
  
  To connect to this Ray cluster:
    import ray
    ray.init(_node_ip_address='0.0.0.0')
  
  To submit a Ray job using the Ray Jobs CLI:
    RAY_ADDRESS='http://0.0.0.0:8265' ray job submit --working-dir . -- python my_script.py
  
  See https://docs.ray.io/en/latest/cluster/running

In [5]:
!docker exec ray-head bash -c "ip addr|grep inet"

    inet 127.0.0.1/8 scope host lo
    inet6 ::1/128 scope host 
    inet 172.17.0.2/16 brd 172.17.255.255 scope global eth0


In [ ]:
SCHEDULER_IP = "172.18.0.2"

#### Start workers

In [ ]:
!docker run -dit --name ray-worker1 test-ray-image ray start --address=ray-head:6379 --block

In [ ]:
!docker run -dit --network simulated-cluster --name ray-worker1 test-ray-image ray start --address=ray-head:6379 --block
!docker run -dit --network simulated-cluster --name ray-worker2 test-ray-image ray start --address=ray-head:6379 --block
!docker run -dit --network simulated-cluster --name ray-worker3 test-ray-image ray start --address=ray-head:6379 --block

In [17]:
!docker run -dit --name ray-worker1 test-ray-image ray start --address=172.17.0.2:6379 --block
!docker run -dit --name ray-worker2 test-ray-image ray start --address=172.17.0.2:6379 --block
!docker run -dit --name ray-worker3 test-ray-image ray start --address=172.17.0.2:6379 --block

892da8a30e20cc2d74e67f08e859820c2b206b720b274df6d2dd49b24f7e53ef
b53d65ad9bff25966cfb467be902adab6a8c716e0b42f9c4fba36a24e9089607
dc4170ae84b655aac05bcb1781d0b9d3222e784521478c2e81c5b1d1b5fbf1b2


In [21]:
!docker logs ray-worker1

Local node IP: 172.17.0.3
2025-03-04 01:36:03,841	WARNING services.py:2072 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=5.02gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
[2025-03-04 01:36:03,872 W 1 1] global_state_accessor.cc:429: Retrying to get node with node ID 93182829376f69e3a7301cba65f37d976b6cdfa271f45371a5601077
[2025-03-04 01:36:04,873 W 1 1] global_state_accessor.cc:429: Retrying to get node with node ID 93182829376f69e3a7301cba65f37d976b6cdfa271f45371a5601077

--------------------
Ray runtime started.
--------------------

To terminate the Ray runtime, run
  ray stop

--block
  This command will now block forever until terminated by a

### Now test the network

In [ ]:
#!pip install "ray[all]"

In [1]:
import ray

In [9]:
ray.init("ray://localhost:6379")

@ray.remote
def test():
    return "Hello from Ray!"

2025-03-03 19:53:17,423	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver


ConnectionError: ray client connection timeout